In [ ]:
!pip install datasets
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 3.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

import transformers
import pickle
import torch
import shap

from datasets import load_dataset, load_metric, Dataset, DatasetDict
from transformers import AutoTokenizer, pipeline
from collections import defaultdict
from tqdm import tqdm
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

print(f"Using '{device}' device")

Using 'cuda' device


In [ ]:
with open("drive/MyDrive/multilabel_emoji_prediction/bert.pkl", 'rb') as f:
    model = pickle.load(f)
model_name  = "google-bert/bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length = 128)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
df_test = pd.read_csv("drive/MyDrive/multilabel_emoji_prediction/balanced_test.csv")
test_dataset = Dataset.from_pandas(df_test)

ROW_NUM = df_test.shape[0]
labels  = df_test.columns[2:]
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in id2label.items( )}

def preprocess_data(examples):
    def tokenize_function(examples, padding="longest", truncation=True):
        return tokenizer(examples["text"], padding = padding, truncation = truncation)

    encoding = tokenize_function(examples)
    batch_size = len(examples["text"])
    num_labels = len(labels)
    label_array = np.zeros((batch_size, num_labels))

    for label_name in labels:
        if label_name in examples:
            j = label2id[label_name]
            for i, val in enumerate(examples[label_name]):
                if val:
                    label_array[i, j] = 1

    encoding['labels'] = label_array

    return encoding

test_preprocessed = test_dataset.map(preprocess_data, batched = True, batch_size = None)

Map:   0%|          | 0/3001 [00:00<?, ? examples/s]

In [ ]:
pred = pipeline(
    "text-classification",
    model = model,
    tokenizer = tokenizer,
    device = device,
    return_all_scores = True,
)
explainer = shap.Explainer(pred)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
# shap_values = explainer(test_preprocessed["text"]) ### 4 hours needed for 3000 rows

with open("drive/MyDrive/multilabel_emoji_prediction/shap_values.pkl", 'rb') as f:
    shap_values = pickle.load(f)

In [ ]:
encoded_list = []

for i in tqdm(range(ROW_NUM)):
    input_text = test_preprocessed['text'][i]
    tokens = tokenizer.encode(input_text, add_special_tokens=True)
    tokens_list = tokenizer.convert_ids_to_tokens(tokens)
    encoded_list.append(tokens_list)

100%|██████████| 3001/3001 [00:23<00:00, 130.18it/s]


In [ ]:
truth_arr = np.load("drive/MyDrive/multilabel_emoji_prediction/truth.npy")
preds_arr = np.load("drive/MyDrive/multilabel_emoji_prediction/preds.npy")

In [ ]:
impact_dict = defaultdict(dict)

def update_case_dict(case_code, i, label_idx):
    impact_list = shap_values[i, :, label_idx].values
    tokens_list = encoded_list[i]
    for j in range(len(tokens_list)):
        token, impact = tokens_list[j], impact_list[j]
        if case_code == "FP":
            false_positive_dict[token].append(impact)
        elif case_code == "FN":
            false_negative_dict[token].append(impact)
        elif case_code == "TP":
            true_positive_dict[token].append(impact)

def generate_sorted_impact(case_dict, flag = True):
    mean_impact = {key: sum(values) / len(values) for key, values in case_dict.items() if len(values) > 1}
    sorted_impact = sorted(mean_impact.items(), key=lambda item: item[1], reverse=flag)
    return sorted_impact

for label in labels:
    print(label)

    true_positive_dict  = defaultdict(list)
    false_positive_dict = defaultdict(list)
    false_negative_dict = defaultdict(list)

    label_idx = label2id[label]

    for i in tqdm(range(ROW_NUM)):
        truth, pred = truth_arr[i][label_idx], preds_arr[i][label_idx]

        if truth and pred: # TP
            update_case_dict("TP", i, label_idx)
        elif not truth and pred: # FP
            update_case_dict("FP", i, label_idx)
        elif truth and not pred: # FN
            update_case_dict("FN", i, label_idx)

    true_positive_sorted_impact  = generate_sorted_impact(true_positive_dict, True)
    false_positive_sorted_impact = generate_sorted_impact(false_positive_dict, True)
    false_negative_sorted_impact = generate_sorted_impact(false_negative_dict, True)

    impact_dict[label]['TP'] = true_positive_sorted_impact
    impact_dict[label]['FP'] = false_positive_sorted_impact
    impact_dict[label]['FN'] = false_negative_sorted_impact

👍


100%|██████████| 3001/3001 [01:16<00:00, 39.28it/s]


🙏


100%|██████████| 3001/3001 [01:38<00:00, 30.37it/s]


🎉


...

100%|██████████| 3001/3001 [00:39<00:00, 75.42it/s]


🎁


100%|██████████| 3001/3001 [01:05<00:00, 45.66it/s]


🙋


100%|██████████| 3001/3001 [00:32<00:00, 91.37it/s]


In [ ]:
file_path = 'impact_dict.pkl'

with open(file_path, 'wb') as f:
    pickle.dump(impact_dict, f)

print("Dict saved to", file_path)

Dict saved to impact_dict.pkl
